In [1]:
# Code Imports
from data_loader_test import DogsDataSet_Test
from data_loader_train import DogsDataSet_Train
from data_transformations import Transformations
from model import DogIdentificationModel
from params import DEVICE, CPU_DEVICE
from model_trainer import trainer
from model_evaluator import evaluator

# Library Imports
import torch

In [ ]:
import pandas as pd
d = pd.read_csv('boxesDF.csv',index_col=0)

## Model Training

In [2]:
# Loading model
model = DogIdentificationModel()

# Freezing the weights of the feature extractor
for param in model.model.features.parameters():
    param.requires_grad = False
    
# Moving to training device
model = model.to(DEVICE)

In [3]:
BATCH_SIZE = 30

In [4]:
# # Defining the validation data loader
# trainData = DogsDataSet_Test(dataType='train')

# # Defining the Validation data loader
# trainLoader = torch.utils.data.DataLoader(trainData, batch_size=BATCH_SIZE,
#                                           shuffle=True, num_workers=4)

# # # Defining the validation data loader
# # validationData = DogsDataSet_Test(dataType='validation')

# # # Defining the Validation data loader
# # validationLoader = torch.utils.data.DataLoader(validationData, batch_size=1,
# #                                           shuffle=True, num_workers=1)

# # Defining the validation data loader
# validationData = DogsDataSet_Test(dataType='validation')

# # Defining the Validation data loader
# validationLoader = torch.utils.data.DataLoader(validationData, batch_size=BATCH_SIZE,
#                                           shuffle=True, num_workers=4)

In [5]:
# Defining the validation data loader
trainData = DogsDataSet_Train(dataType='train')

# Defining the Validation data loader
trainLoader = torch.utils.data.DataLoader(trainData, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=4)

# # Defining the validation data loader
# validationData = DogsDataSet_Test(dataType='validation')

# # Defining the Validation data loader
# validationLoader = torch.utils.data.DataLoader(validationData, batch_size=1,
#                                           shuffle=True, num_workers=1)

# Defining the validation data loader
validationData = DogsDataSet_Train(dataType='validation')

# Defining the Validation data loader
validationLoader = torch.utils.data.DataLoader(validationData, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=4)

In [6]:
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01,
                            momentum=0.9, weight_decay=0.0005)

# Defining the learning rate that makes a step every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=4,
                                               gamma=0.1)

In [7]:
NUM_EPOCHS = 50
START_EPOCH = 0

In [8]:
BEST_LOSS = 100000000
trainMeanLosses = []
validationMeanLosses = []

In [9]:
# checkpoint = torch.load('dog-identification-model.pt')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# START_EPOCH = checkpoint['epoch'] + 1
# BEST_LOSS = checkpoint['loss']
# trainMeanLosses = checkpoint['trainMeanLosses']
# validationMeanLosses = checkpoint['validationMeanLosses']

In [10]:
for epoch in range(START_EPOCH, NUM_EPOCHS):
    # ***************** TRAINING ******************    

    trainLoss = trainer(model, optimizer, trainLoader, epoch)
    
    trainMeanLosses.append(trainLoss)
    
    # Updating the learning rate scheduler
    lr_scheduler.step()
    
    # ***************** EVALUATION ******************    
    
    validationLoss = evaluator(model, validationLoader, epoch)

    validationMeanLosses.append(validationLoss)
    
    # ***************** SAVING CHECKPOINT ******************    
    
    if validationLoss < BEST_LOSS:
        print('Saving New Model')
        
        PATH = "dog-identification-model-triplet.pt"
        
        torch.save(
            {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': validationLoss,
                'trainMeanLosses': trainMeanLosses,
                'validationMeanLosses': validationMeanLosses
            }, PATH)
        
        BEST_LOSS = validationLoss


EPOCH: 0 | Loss 0.6633735957898592: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [05:12<00:00,  1.40s/it]

EPOCH: 0 | Final Training Mean Loss 0.6708740593341497



EPOCH: 0 | Loss 0.6570906866164434: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:23<00:00,  1.40s/it]


EPOCH: 0 | Final Validation Mean Loss 0.6629945789268631
Saving New Model


EPOCH: 1 | Loss 0.659873510661878: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [05:24<00:00,  1.46s/it]

...

EPOCH: 48 | Final Validation Mean Loss 0.6542815819471895



EPOCH: 49 | Loss 0.6376850228560599: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [05:52<00:00,  1.58s/it]

EPOCH: 49 | Final Training Mean Loss 0.6474480913827333



EPOCH: 49 | Loss 0.6444845653715587: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:27<00:00,  1.62s/it]

EPOCH: 49 | Final Validation Mean Loss 0.6545822615632991


In [20]:
import pandas as pd
pd.DataFrame(trainMeanLosses).to_csv('triplet-train-losses.csv')

In [ ]:
model = model.eval()

In [ ]:
data = iter(validationLoader)

In [ ]:
index, negativeImgIndex, positiveImg, anchorImg, negativeImg = next(data)

In [ ]:
img1, img2, label = next(data)

In [ ]:
img1 = img1.to(DEVICE)
img2 = img2.to(DEVICE)

In [ ]:
img1Encoding = model(img1)
img2Encoding = model(img2)

In [ ]:
from triplet_loss import sigmoidL2
distance = sigmoidL2(img1Encoding,img2Encoding)

In [ ]:
distance[label.reshape(30) == 1]

In [ ]:
distance[label.reshape(30) == 0]

In [ ]:
label.reshape(30)

In [ ]:
distance[29]

In [ ]:
from model_utils import plot_tensor

In [ ]:
plot_tensor(img1[11].cpu())

In [ ]:
plot_tensor(img2[11].cpu())

In [ ]:
label.dtypes

In [ ]:
plot_tensor(positiveImg[4].cpu())

In [ ]:
plot_tensor(anchorImg[4].cpu())

In [ ]:
plot_tensor(negativeImg[4].cpu())

In [ ]:
positiveImg.shape

In [ ]:
positiveImg = positiveImg.to(DEVICE)
anchorImg = anchorImg.to(DEVICE)
negativeImg = negativeImg.to(DEVICE)

In [ ]:
positiveImgEncoding = model(positiveImg)
anchorImgEncoding = model(anchorImg)
negativeImgEncoding = model(negativeImg)

In [ ]:
from triplet_loss import sigmoidL2, triplet_loss

In [ ]:
triplet_loss(anchorImgEncoding, positiveImgEncoding, negativeImgEncoding, 0.9)

In [ ]:
sigmoidL2(positiveImgEncoding, anchorImgEncoding)[0]

In [ ]:
sigmoidL2(negativeImgEncoding, anchorImgEncoding)

In [ ]:
positiveImgEncoding.shape